In [168]:
# Libraries
import numpy as np   
import time
import scipy.sparse.linalg as spy
import warnings
warnings.filterwarnings('ignore')


def pos_def(n,x=5):
    A = np.random.rand(n,n)
    return (A+A.T) + x*np.eye(n)

n = 100
A = pos_def(n,x=8)
x_sol = np.floor(np.random.rand(n) * 100)
b = np.dot(A, x_sol)
print np.linalg.cond(A)
print (np.linalg.eig(A)[0] > 0).all()

269.891314066
True


In [160]:
# A better implementation of preconditioned GMRes without Cauchy integral (left)
def prec_gmres(A_0,b_0,alpha,it=100,tol=1e-6,left = True):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    # copy
    A = np.copy(A_0)
    b = np.copy(b_0)
    M = A + alpha*np.identity(b.shape[0])
    
    # Left preconditioner -> A and b changes
    if left:
        b = gmres(M,b,tol=tol)[0]
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        if left:
            w = gmres(M, np.dot(A,Q[:,i]),tol=tol)[0]
        else:
            w = np.dot(A, gmres(M, Q[:,i],tol=tol)[0])
        
        for j in range(i+1):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,_,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        residual = np.linalg.norm(np.dot(H[:i+2,:i+1],y) - beta1*e)
        
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    if left:
        return x_tild, i+1
    else:
        return gmres(M, x_tild,tol=tol)[0],i+1

In [161]:
def trapezoid2(myfun, N, a, b):
    x = np.linspace(0, b, N/2) # We want N bins, so N+1 points  
    h = x[1]-x[0]
    xmiddle = x[1:-1]
    int_val = 0
    for i in xmiddle:
        int_val += 2*myfun(i).real
    int_val = 2*myfun(a).real + 2*int_val + 2*myfun(0)# + myfun(b)
    return 0.5*h*int_val

def z(t, c, r):
    return c + r*np.complex(np.cos(t), np.sin(t))

def dz(t, r):
    return r*np.complex(np.cos(t), np.sin(t))

def g(t,l,L,alpha,v,f, tol=1e-6):
    centro = (l + L)/2.
    radio = (L - l)/2.
    fz = f(z(t, centro, radio))
    dzz = dz(t, radio)
    p = fz*dzz
    # Separamos las matrices
    M = (centro + dzz - alpha)*np.identity(v.shape[0]) - A
    a = M.real
    b = M.imag
    al = v.real
    bet = v.imag
    # Construímos el nuevo sistema Hx = r
    H = np.zeros((a.shape[0]*2, a.shape[1]*2))
    H[:a.shape[0],:a.shape[1]] = a
    H[a.shape[0]:,a.shape[1]:] = a
    H[:a.shape[0],a.shape[1]:] = -b
    H[a.shape[0]:,:a.shape[1]] = b
    r = np.zeros((al.shape[0]*2))
    r[:al.shape[0]] = al
    r[al.shape[0]:] = bet
    
    sol = gmres(H,r,tol=tol)[0]
    gmr = sol[:(sol.shape[0]/2)] + 1j*sol[(sol.shape[0]/2):]
    #gmr = p*spy.gmres((centro + dzz - alpha)*np.identity(v.shape[0]) - A, v)[0]
    return p*gmr


def cauchy_integral(l, L, alpha, v, Nf = 50,N=128,tol=1e-6):
    f = lambda x: (1. - (alpha/x)**(Nf+1))/(x - alpha)
    g1 = lambda t: g(t,l,L,alpha,v,f,tol=tol)
    val = trapezoid2(g1, N, -np.pi, np.pi) / (2.*np.pi)
    return val

In [165]:
# GMRes using contour integral to compute the preconditioner
def gmres(A_0, b_0, it=100, tol=1e-6, prec=False, left=True, alpha=0.0,aux_tol=1e-6):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    A = np.copy(A_0)
    b = np.copy(b_0)
    
    if prec:
        # Cambiar a cotas
        lamb, vect = np.linalg.eig(A)
        L = np.amax(np.real(lamb))
        l = np.amin(np.real(lamb))
        d = np.amax(np.imag(lamb))
        l = l + alpha - l/2
        L = L + alpha + L/2
        if left:
            b = cauchy_integral(l, L, alpha, b,tol=aux_tol)
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        if prec:
            if left:
                w = cauchy_integral(l,L,alpha,np.dot(A,Q[:,i]),tol=aux_tol)
            else:
                w = np.dot(A, cauchy_integral(l, L, alpha, Q[:,i],tol=aux_tol))
        else:
            w = np.dot(A,Q[:,i])
        
        for j in range(i+1):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,_,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        residual = np.linalg.norm(np.dot(H[:i+2,:i+1],y) - beta1*e)
        
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    if left or not prec:
        return x_tild,i+1
    else:
        return cauchy_integral(l, L, alpha, x_tild,tol=aux_tol),i+1

In [169]:
alpha = 0.05
print "Exact solution:\n", x_sol
print "\nNp Solve:\n", spy.gmres(A,b)
print "\nNormal GMRes:\n", gmres(A,b)
print "\nLeft Prec NC:\n", prec_gmres(A,b,alpha)
#print "\nRight Prec NC:\n", prec_gmres(A,b,alpha,left=False)
print "\nCauchy Left Prec:\n", gmres(A, b,prec=True,alpha=alpha)
#print "\nCauchy Right Prec:\n", gmres(A,b,prec=True,left=False,alpha=alpha)

Exact solution:
[ 89.  55.  11.  95.  50.  71.  60.   6.  62.  11.  48.  29.   9.  75.   5.
  45.  68.  46.  26.  98.  21.  69.  99.  96.  75.  88.  95.  14.   8.  48.
  35.  27.  60.  78.  98.  63.  36.  52.  68.  51.  14.   6.  25.   3.  18.
  91.   7.  60.  98.  41.  12.  42.  22.  69.  68.  75.  45.  28.  33.  77.
  65.  36.  51.  55.  10.   0.  58.  17.  62.  51.  55.  62.  77.  10.  89.
  66.   2.  72.  95.  45.  20.  97.   5.  94.  27.  87.  83.   2.  74.  55.
  31.  21.  11.  82.  79.  62.  81.  15.   9.  91.]

Np Solve:
(array([  8.91649139e+01,   5.50184052e+01,   1.10078140e+01,
         9.49743897e+01,   5.00192734e+01,   7.09773100e+01,
         5.99947858e+01,   6.08063847e+00,   6.19283094e+01,
         1.09577200e+01,   4.79676255e+01,   2.89447924e+01,
         8.99382865e+00,   7.50716502e+01,   5.02750598e+00,
         4.50504278e+01,   6.80460881e+01,   4.58552708e+01,
         2.59701524e+01,   9.79848733e+01,   2.10556774e+01,
         6.88672370e+01,   9.89638873